In [951]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
import sklearn
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
pd.options.display.max_columns = 100

In [952]:
df=pd.read_csv("parsed_df.csv")
del df['Unnamed: 0']
del df['ACTION_TYPE']
del df['PLAYER_ID']

In [953]:
#drop all rows with NaN values
df = df.dropna()

In [954]:
cols = list(df)
cols.insert(0, cols.pop(cols.index('SHOT_MADE_FLAG')))
df = df.ix[:, cols]

In [955]:
dftouse=df.copy()
dftouse=dftouse.dropna()
dftouse=dftouse.sample(30000, replace=False)

In [956]:
dftouse.shape

(30000, 76)

In [957]:
#dftouse.isnull().sum()

In [958]:
#dftouse.isnull().any()

In [963]:
Xnames = list(dftouse.columns.values[1:76])

In [964]:
Xnames.remove('PLAYER_NAME')

In [965]:
Xnames

['EVENTNUM',
 'PERIOD_x',
 'SCOREMARGIN',
 'ACTION_BEFORE',
 'CLOSE_DEF_DIST',
 'DRIBBLES',
 'LOCATION',
 'PTS_TYPE',
 'SHOT_CLOCK',
 'SHOT_NUMBER',
 'TOUCH_TIME',
 'GAME_EVENT_ID',
 'SHOT_DISTANCE',
 'LOC_X',
 'LOC_Y',
 'YEAR',
 'SEC_LEFT_Q',
 'SEC_LEFT_G',
 'OFF_HEIGHT',
 'DEF_HEIGHT',
 'HEIGHT_DIFF',
 'SHOT_DISTxCLOSE_DEF_DIST',
 'HEIGHTxCLOSE_DEF_DIST',
 'SHOT_DISTxJUMP_SHOT',
 'NO_DRIBBLE',
 'Alley Oop Layup shot',
 'Driving Bank Hook Shot',
 'Driving Bank shot',
 'Driving Dunk Shot',
 'Driving Finger Roll Layup Shot',
 'Driving Hook Shot',
 'Driving Jump shot',
 'Driving Layup Shot',
 'Driving Reverse Layup Shot',
 'Driving Slam Dunk Shot',
 'Dunk Shot',
 'Fadeaway Bank shot',
 'Fadeaway Jump Shot',
 'Finger Roll Layup Shot',
 'Floating Jump shot',
 'Hook Bank Shot',
 'Hook Shot',
 'Jump Bank Hook Shot',
 'Jump Bank Shot',
 'Jump Hook Shot',
 'Jump Shot',
 'Layup Shot',
 'Pullup Bank shot',
 'Pullup Jump shot',
 'Putback Dunk Shot',
 'Putback Layup Shot',
 'Putback Reverse Dunk S

In [966]:
#create train and test set
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [967]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5, n_jobs=1):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    y_=indf[targetname]
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [948]:
clfForest = RandomForestClassifier()
parameters = {
    "n_estimators" : [10,50,100,500,1000],
    #"n_estimators" : range(1, 21),
    #"max_features" : range(10,len(Xnames+1),2)
    "max_depth": [100,200,400,800],
    #"bootstrap": [True, False]
    }

In [949]:
%%time
clfForest, Xtrain, ytrain, Xtest, ytest = do_classify(clfForest, parameters, dftouse, Xnames, 'SHOT_MADE_FLAG', 1, mask=mask, n_folds = 5, n_jobs = 4, score_func='f1')

using mask


/Users/raymondcen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored

KeyboardInterrupt


In [950]:
importance_list = clfForest.feature_importances_
name_list = dftouse.columns
importance_list, name_list = zip(*sorted(zip(importance_list, Xnames)))
#full list of features
#ax = sns.barplot(y=range(len(name_list)),x=list(reversed(importance_list)), orient='h', palette="Blues_d")
sns.set_context("paper", font_scale=1.5)
plt.figure(figsize=(12, 9))
#top 20 features
ax = sns.barplot(y=range(20),x=list(reversed(importance_list))[:20], orient='h', palette="Blues_d")
ax.set_xlabel('Relative Importance in the Random Forest')
ax.set_ylabel('Features')
ax.set_title('Shot Made: Relative Importance of Each Feature')
ax.set_yticklabels(list(reversed(name_list)));
sns.despine()
plt.savefig('Baseline Relative Importance.png', bbox_inches='tight')

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [924]:
df2013 = df[df['YEAR']==2013]
df2013.reset_index(drop=True, inplace=True)
Xtrain2013 = df2013.ix[:, 1:76]
ytrain2013 = df2013['SHOT_MADE_FLAG']

In [925]:
%%time
clfForest.fit(Xtrain2013, ytrain2013)

CPU times: user 4min 29s, sys: 3.65 s, total: 4min 32s
Wall time: 4min 33s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=200, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [968]:
df2014 = df[df['YEAR']==2014]
Xtest2014 = df2014.ix[:, 1:76]
df2014.head()

,SHOT_MADE_FLAG,EVENTNUM,PERIOD_x,SCOREMARGIN,ACTION_BEFORE,CLOSE_DEF_DIST,DRIBBLES,LOCATION,PTS_TYPE,SHOT_CLOCK,SHOT_NUMBER,TOUCH_TIME,GAME_EVENT_ID,PLAYER_NAME,SHOT_DISTANCE,LOC_X,LOC_Y,YEAR,SEC_LEFT_Q,SEC_LEFT_G,OFF_HEIGHT,DEF_HEIGHT,HEIGHT_DIFF,SHOT_DISTxCLOSE_DEF_DIST,HEIGHTxCLOSE_DEF_DIST,SHOT_DISTxJUMP_SHOT,NO_DRIBBLE,Alley Oop Layup shot,Driving Bank Hook Shot,Driving Bank shot,Driving Dunk Shot,Driving Finger Roll Layup Shot,Driving Hook Shot,Driving Jump shot,Driving Layup Shot,Driving Reverse Layup Shot,Driving Slam Dunk Shot,Dunk Shot,Fadeaway Bank shot,Fadeaway Jump Shot,Finger Roll Layup Shot,Floating Jump shot,Hook Bank Shot,Hook Shot,Jump Bank Hook Shot,Jump Bank Shot,Jump Hook Shot,Jump Shot,Layup Shot,Pullup Bank shot,Pullup Jump shot,Putback Dunk Shot,Putback Layup Shot,Putback Reverse Dunk Shot,Putback Slam Dunk Shot,Reverse Dunk Shot,Reverse Layup Shot,Reverse Slam Dunk Shot,Running Bank Hook Shot,Running Bank shot,Running Dunk Shot,Running Finger Roll Layup Shot,Running Hook Shot,Running Jump Shot,Running Layup Shot,Running Reverse Layup Shot,Running Slam Dunk Shot,Running Tip Shot,Slam Dunk Shot,Step Back Jump shot,Tip Shot,Turnaround Bank Hook Shot,Turnaround Bank shot,Turnaround Fadeaway shot,Turnaround Hook Shot,Turnaround Jump Shot
200963,0,2,1,0,10,4.6,0,1,2,10.6,1,0.9,2,Anthony Davis,20,50,194,2014,703,2863,82,82,0,92.0,0.0,20,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
200964,1,4,1,2,4,6.5,0,0,2,15.4,1,1.1,4,Nikola Vucevic,18,-8,189,2014,691,2851,84,84,0,117.0,0.0,0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
200965,0,7,1,-2,5,11.6,0,1,2,13.5,2,1.0,7,Anthony Davis,18,-131,127,2014,666,2826,82,84,-2,208.8,-23.2,18,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
200966,0,9,1,2,4,2.0,11,0,2,13.7,1,8.8,9,Elfrid Payton,1,-15,4,2014,654,2814,76,76,0,2.0,0.0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
200967,1,25,1,1,5,7.7,1,1,2,21.6,3,3.7,25,Anthony Davis,0,0,1,2014,629,2789,82,82,0,0.0,0.0,0,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [929]:
%%time
predictions = clfForest.predict_proba(Xtest2014)

CPU times: user 37.3 s, sys: 7.58 s, total: 44.9 s
Wall time: 52.7 s


In [930]:
predictions

array([[ 0.942,  0.058],
       [ 0.194,  0.806],
       [ 0.708,  0.292],
       ..., 
       [ 0.84 ,  0.16 ],
       [ 0.618,  0.382],
       [ 0.474,  0.526]])

In [973]:
MSE = [(x-y)**2 for x,y in zip(df2014.SHOT_MADE_FLAG,df2014.pred)]

In [975]:
np.average([1 if (x-.5)*(y-.5)>0 else 0 for x,y in zip(df2014.SHOT_MADE_FLAG,df2014.pred)])

0.66208887799784788

In [969]:
df2014['pred']=[x[1] for x in predictions]
df2014['resid']=(df2014.SHOT_MADE_FLAG-df2014.pred)*df2014.PTS_TYPE
players=list(set(df2014['PLAYER_NAME']))
pct_above=[]
predpps=[]
count=[]
players_act=[]
for player in players:
    subset=df2014[df2014['PLAYER_NAME']==player]
    if len(subset.index)>100:
        players_act.append(player)
        pct_above.append(np.average(subset.resid))
        predpps.append(np.average(subset.pred*subset.PTS_TYPE))
        count.append(len(subset.index))

/Users/raymondcen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/raymondcen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [977]:
comb=zip(players_act,pct_above,predpps,count)
comb.sort(key=lambda tup: tup[1],reverse=True)
comb

[('Luke Babbitt', 0.27706077348066305, 0.99918232044198896, 181),
 ('Kyle Korver', 0.20435081374321881, 1.1265714285714286, 553),
 ('Andre Miller', 0.19285714285714287, 0.8380308880308881, 259),
 ('Damjan Rudez', 0.17541035856573708, 0.99988844621513939, 251),
 ('Ed Davis', 0.16905699481865286, 1.1003730569948187, 386),
 ('Chris Paul', 0.15882481751824817, 0.94792700729926993, 1096),
 ('George Hill', 0.15221487603305783, 0.93869421487603311, 484),
 ('CJ Watson', 0.14560629921259843, 0.94625721784776906, 381),
 ('Al Jefferson', 0.14522048066875654, 0.83597074190177634, 957),
 ('Carl Landry', 0.13496296296296295, 0.91347008547008546, 351),
 ('TJ Warren', 0.13383168316831684, 0.98993069306930692, 202),
 ('Hedo Turkoglu', 0.13076923076923075, 1.0890109890109889, 182),
 ('Ben Gordon', 0.12246441947565544, 0.91498876404494389, 267),
 ('Anthony Morrow', 0.12058782608695651, 1.0568034782608697, 575),
 ('Jose Calderon', 0.11462130177514794, 0.86466863905325442, 338),
 ('Darren Collison', 0.1143